# NCAA Tournament Simulations

This notebook simulates the 2015 NCAA Tournament using our distribution of features.

## Setup

In [47]:
# Libraries.
import numpy as np
import pandas as pd
import pickle

# Custom libraries.
import game_predictions
# game_predictions = reload(game_predictions)

# Library settings.
% matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [42]:
# Paths.
# Pre-tournament statistics path.
stats_path = '../../../data/kenpom_summaries/summary15_pt.csv'
# Bracket paths.
tournament_bracket_path = '../../../data/tournament_brackets/2015_64.csv'
final_four_bracket_path = '../../../data/tournament_brackets/2015_4.csv'

## Data

In [14]:
# Read stats.
pre_tournament_stats = pd.read_csv(stats_path)

In [43]:
# Read brackets.
tournament_bracket = pd.read_csv(bracket_path, header=None).values
final_four_bracket = pd.read_csv(final_four_bracket_path, header=None).values

In [79]:
# Slice trace.
with open('slice_samples_traces.pkl', 'rb') as fid:
    slice_trace = pickle.load(fid)
# Metropolis-Hastings trace.
with open('MH_traces.pkl', 'rb') as fid:
    mh_trace = pickle.load(fid)

In [11]:
# Features.
features = ['diff_Pythag','diff_AdjOE','diff_AdjDE','location_Away','diff_RankAdjOE']

## Simulations

### Slice

#### Whole-Tournament

In [80]:
# Simulate matchups and outcomes.
slice_tournament_matchups, slice_tournament_outcomes = game_predictions.simulate_tournament(tournament_bracket, pre_tournament_stats, features, coef_trace=slice_trace)

In [81]:
slice_tournament_matchups.head()

,winner,loser,round_of,count
0,Arizona,Baylor,16,9998
1,Arizona,Gonzaga,2,2
2,Arizona,Iowa St.,2,1
3,Arizona,Kentucky,4,9
4,Arizona,Ohio St.,32,6941


In [82]:
slice_tournament_matchups[slice_tournament_matchups.round_of == 2].sort('count', ascending=False).head()

,winner,loser,round_of,count
82,Kentucky,Villanova,2,2945
73,Kentucky,Gonzaga,2,1693
72,Kentucky,Duke,2,1558
83,Kentucky,Virginia,2,1013
190,Wisconsin,Villanova,2,829


In [83]:
slice_tournament_outcomes

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Albany,0,0,0,0,0,0
Arizona,10000,10000,10000,3216,9,5
Arkansas,10000,4,0,0,0,0
Baylor,10000,9998,0,0,0,0
Belmont,0,0,0,0,0,0
Buffalo,92,40,0,0,0,0
Butler,4850,29,7,0,0,0
Cincinnati,5034,0,0,0,0,0
Coastal Carolina,0,0,0,0,0,0
Davidson,5541,0,0,0,0,0


In [84]:
slice_tournament_outcomes.sort('wins_round_6', ascending=False).head(n=7)

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Kentucky,10000,10000,10000,9988,7343,7210
Wisconsin,10000,10000,10000,6784,2648,2173
Villanova,10000,10000,10000,7926,4356,581
Duke,10000,9994,9409,4803,2256,22
Virginia,10000,9914,9914,2074,1110,7
Arizona,10000,10000,10000,3216,9,5
Gonzaga,10000,10000,9772,4976,2276,2


#### Final Four

In [85]:
# Simulate matchups and outcomes.
slice_final_four_matchups, slice_final_four_outcomes = game_predictions.simulate_tournament(final_four_bracket, pre_tournament_stats, features, coef_trace=slice_trace)

In [87]:
slice_final_four_matchups

,winner,loser,round_of,count
0,Duke,Kentucky,2,42
1,Duke,Michigan St.,4,10000
2,Duke,Wisconsin,2,8
3,Kentucky,Duke,2,6315
4,Kentucky,Wisconsin,4,6357
5,Wisconsin,Duke,2,3635
6,Wisconsin,Kentucky,4,3643


In [88]:
slice_final_four_outcomes

,wins_round_1,wins_round_2
Duke,10000,50
Kentucky,6357,6315
Michigan St.,0,0
Wisconsin,3643,3635


### Metropolis-Hastings

#### Whole-Tournament

In [56]:
# Simulate matchups and outcomes.
mh_tournament_matchups, mh_tournament_outcomes = game_predictions.simulate_tournament(tournament_bracket, pre_tournament_stats, features, coef_trace=mh_trace)

In [68]:
mh_tournament_matchups.head()

,winner,loser,round_of,count
0,Arizona,Baylor,16,9973
1,Arizona,Duke,2,43
2,Arizona,Gonzaga,2,70
3,Arizona,Iowa St.,2,2
4,Arizona,Kentucky,4,435


In [69]:
mh_tournament_matchups[mh_tournament_matchups.round_of == 2].sort('count', ascending=False).head()

,winner,loser,round_of,count
92,Kentucky,Villanova,2,2433
93,Kentucky,Virginia,2,2242
82,Kentucky,Gonzaga,2,1946
81,Kentucky,Duke,2,1212
200,Wisconsin,Gonzaga,2,321


In [70]:
mh_tournament_outcomes

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Albany,0,0,0,0,0,0
Arizona,10000,10000,9999,4826,435,246
Arkansas,10000,27,0,0,0,0
Baylor,10000,9974,1,0,0,0
Belmont,0,0,0,0,0,0
Buffalo,63,34,0,0,0,0
Butler,4975,258,72,0,0,0
Cincinnati,5391,0,0,0,0,0
Coastal Carolina,0,0,0,0,0,0
Davidson,4651,1,1,0,0,0


In [72]:
mh_tournament_outcomes.sort('wins_round_6', ascending=False).head(n=7)

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Kentucky,10000,10000,10000,9977,8035,7862
Wisconsin,10000,10000,10000,5174,1529,1184
Villanova,10000,10000,10000,5562,3245,481
Arizona,10000,10000,9999,4826,435,246
Virginia,10000,9966,9966,4438,2698,126
Duke,10000,9934,6870,3594,1637,65
Gonzaga,10000,9999,9583,4879,2373,36


#### Final Four

In [52]:
# Simulate matchups and outcomes.
mh_final_four_matchups, mh_final_four_outcomes = game_predictions.simulate_tournament(final_four_bracket, pre_tournament_stats, features, coef_trace=mh_trace)

In [75]:
mh_final_four_matchups

,winner,loser,round_of,count
0,Duke,Kentucky,2,46
1,Duke,Michigan St.,4,10000
2,Duke,Wisconsin,2,185
3,Kentucky,Duke,2,7118
4,Kentucky,Wisconsin,4,7164
5,Wisconsin,Duke,2,2651
6,Wisconsin,Kentucky,4,2836


In [76]:
mh_final_four_outcomes

,wins_round_1,wins_round_2
Duke,10000,231
Kentucky,7164,7118
Michigan St.,0,0
Wisconsin,2836,2651
